In [ ]:
import torch

from lib.models.temporal_predictor import TemporalPredictor


def load_resnet18_temporal_model(checkpoint):
    model = TemporalPredictor("resnet18",
                              dim=512,
                              hidden_dim=128)
    data = torch.load(checkpoint, map_location="cpu")

    model.load_state_dict(data["model_dict"])
    #model = model.backbone
    print(f"Loaded ResNet18 model from {checkpoint}")
    print(f"{sum(p.numel() for p in model.parameters()):,} parameters")
    model.eval()

    return model

model = load_resnet18_temporal_model("models/temporal/temporal_18_beta_detach/checkpoint_90636.pth")

model = model.to("cuda:0")

In [9]:
from pyphoon2.DigitalTyphoonDataset import DigitalTyphoonDataset as DTD
from torchvision import transforms as T
from lib.utils.fisheye import FishEye

transforms = T.Compose([
    T.ToTensor(),
    FishEye(256, 0.2),
])

def transform_func(obj):
    img, labels = obj
    img_range = [150, 350]
    img, labels = obj
    img = (img - img_range[0])/(img_range[1]-img_range[0])

    return transforms(img.astype(np.float32)), labels


dataset = DTD(image_dir="/fs9/gaspar/data/WP/image/",
              metadata_dir="/fs9/gaspar/data/WP/metadata/",
              metadata_json="/fs9/gaspar/data/WP/metadata.json",
              get_images_by_sequence=True,
              labels=("year", "month", "day", "hour", "grade"),#("grade", "pressure", "wind"),
              split_dataset_by="sequence",
              load_data_into_memory='track',
              filter_func= lambda x: x.grade() > 2 and x.grade() < 6,
              transform=None,
              ignore_list=[],
              verbose=False)

In [ ]:
import numpy as np
from torchvision import transforms as T
from lib.utils.fisheye import FishEye
from lib.utils.dataset import PreprocessedTyphoonDataset

transforms = T.Compose([
    #T.ToTensor(),
    FishEye(256,0.2),
])

def transform_func(obj):
    img_range = [150, 350]
    seq= obj
    seq = (seq - img_range[0])/(img_range[1]-img_range[0])
    seq = torch.from_numpy(seq.astype(np.float32))
    seq = seq.unsqueeze(1)
    return transforms(seq)#, labels


dataset = PreprocessedTyphoonDataset(image_dir="/fs9/gaspar/data/WP/image/",
              metadata_dir="/fs9/gaspar/data/WP/metadata/",
              metadata_json="/fs9/gaspar/data/WP/metadata.json",
              labels=("grade", "pressure", "wind"),
              split_dataset_by="sequence",
              load_data_into_memory='track',
              filter_func= lambda x: x.grade() < 6 and x.year() > 1990,
              transform=None,
              ignore_list=[],
              verbose=False,
              feature_extractor=model
              )

In [21]:
transforms = T.Compose([
    T.ToTensor(),
    FishEye(256,0.2),
])

def transform_func(img):
    img_range = [150, 350]
    img = (img - img_range[0])/(img_range[1]-img_range[0])
    print(img.shape)
    return transforms(img.astype(np.float32))

seq, labels = dataset[540]
print(seq.shape)

idx1 = 32
idx2 = 34
device = "cuda:0"
img1, img2 = transform_func(seq[idx1]).to(device), transform_func(seq[idx2]).to(device)

with torch.no_grad():
    print(labels[idx1])
    print(labels[idx2])
    print(img2.shape)
    print(f"Prediction: {model(img1.unsqueeze(0), img2.unsqueeze(0))}")

(129, 512, 512)
(512, 512)
(512, 512)
[2000    7    4   16    5]
[2000    7    4   18    5]
torch.Size([1, 256, 256])
Prediction: tensor([[0.7838]], device='cuda:0')


In [1]:
from lib.utils.dataset import SequenceTyphoonDataset as STD
from torch.utils.data import DataLoader
import numpy as np
import random
import torch

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

dataset = STD(labels=["month", "day", "hour", "pressure", "wind"],
              include_images=False,
              x=[0,1,2,3,4],
              y=[3,4],
              num_inputs=12,
              num_preds=1,
              filter_func= lambda x: x.grade() < 6,)
train, val, test = dataset.random_split([0.7, 0.15, 0.15], split_by="sequence")

print(f"\n{len(train)} train sequences")
print(f"{len(val)} val sequences")
print(f"{len(test)} test sequences")

test_loader = DataLoader(test,
                        batch_size=1,
                        shuffle=False,
                        num_workers=0)


[766, 164, 163] 1093
1093

766 train sequences
164 val sequences
163 test sequences


In [4]:
from lib.models.lstm_predictor import LSTM

def _load_checkpoint(model, path):
    data = torch.load(path)
    model.load_state_dict(data["model_dict"])

    print("="*100)
    print(f"Loading model from checkpoint {path}")
    print("="*100)

    return model

model = LSTM(
    69,
    hidden_size=512,
    num_layers=2,
    output_size=2
)

model = _load_checkpoint(model, "/fs9/gaspar/forecoon/models/ts/biglstm/checkpoint_1200.pth")



Loading model from checkpoint /fs9/gaspar/forecoon/models/ts/biglstm/checkpoint_1200.pth


In [8]:
loader=iter(test_loader)


In [12]:
labels = next(loader)[0]
print(labels.shape)


torch.Size([271, 69])


In [5]:
from lib.utils.dataset import NORMALIZATION

pressure = lambda x: x*NORMALIZATION['pressure'][1]+NORMALIZATION['pressure'][0]
wind = lambda x: x*NORMALIZATION['wind'][1]+NORMALIZATION['wind'][0]

def evaluate_results(ys, preds, verbose=False):
    ys = np.array([pressure(ys[0]), wind(ys[1])])
    preds = np.array([pressure(preds[0]), wind(preds[1])])
    diff = np.abs(preds-ys)
    #diff_rel = diff/ys

    if verbose:
        print(f"Pressure| true: {ys[0]:.3f} pred: {preds[0]:.3f} | diff: {diff[0]:.3f}")# ({diff_rel[0]*100:.3f}%)")
        print(f"Wind|\t  true: {ys[1]:.3f} pred: {preds[1]:.3f} | diff: {diff[1]:.3f}")# ({diff_rel[1]*100:.3f}%)")

    return diff#, diff_rel


In [28]:

def retrieve_labels(vec):
    print(f"Pressure: {vec[0]*NORMALIZATION['pressure'][1]+NORMALIZATION['pressure'][0]}")
    print(f"Wind: {vec[1]*NORMALIZATION['wind'][1]+NORMALIZATION['wind'][0]}")
    #print(f"Grade: {torch.argmax(vec[2:])}")


start = 60
ins = labels[start:start+12, test_loader.dataset.dataset.x].unsqueeze(0)
print(labels[start:start+13,test_loader.dataset.dataset.y])
outs = labels[start+12, test_loader.dataset.dataset.y]
with torch.no_grad():
    evaluate_results(outs.cpu(), model(ins).squeeze().cpu(), True)

tensor([[-0.8356,  1.0183],
        [-0.9111,  1.0183],
        [-0.9822,  1.0183],
        [-1.0578,  1.0183],
        [-1.1333,  1.0183],
        [-1.2044,  1.0183],
        [-1.2800,  1.1713],
        [-1.3156,  1.1713],
        [-1.3556,  1.1713],
        [-1.3911,  1.1713],
        [-1.4267,  1.1713],
        [-1.4667,  1.1713],
        [-1.5022,  1.3242]], device='cuda:0')
Pressure| true: 950.000 pred: 949.449 | diff: 0.551
Wind|	  true: 80.000 pred: 77.272 | diff: 2.728


In [ ]:
from tqdm import tqdm

errors = []

for seq in tqdm(test_loader):
    labels = seq[0]
    for start in range(len(labels) - 13):
        ins = labels[start:start+12, test_loader.dataset.dataset.x].unsqueeze(0)
        outs = labels[start+12, test_loader.dataset.dataset.y]
        with torch.no_grad():
            errors.append(evaluate_results(outs, model(ins).squeeze()))

pressure, wind = zip(*errors)
pressure = np.mean(pressure)
wind = np.mean(wind)
print(pressure, wind)

In [ ]:
from copy import deepcopy

@torch.no_grad()
def forecast(model, labels, start, num_preds, ws = 12):
    labs = labels[start:start+ws, test_loader.dataset.dataset.x]
    errors = []
    for i in range(num_preds):
        #print(labs.shape)
        ins = labs[-ws:]
        outs = labels[start+i+ws, test_loader.dataset.dataset.y].unsqueeze(0)
        preds = model(ins.unsqueeze(0)).squeeze()
        # print(outs, preds)
        errors.append(evaluate_results(outs.squeeze().cpu(), preds.cpu(), False))
        new_data = deepcopy(labels[start+i+ws, test_loader.dataset.dataset.x])
        new_data[test_loader.dataset.dataset.y] = preds

        labs = torch.cat((labs, new_data.unsqueeze(0)))
    return np.array(errors)

forecast(model, labels, 30, 12)

In [14]:
from tqdm import tqdm

ws = 12
errors = []
model = model.to("cuda")
for seq in tqdm(test_loader):
    labels = seq[0].to("cuda")
    for start in range(len(labels) - (ws*2)):
        errors.append(forecast(model, labels, start, ws, ws))
errors = np.array(errors)

print(np.mean(errors, axis=0))



100%|██████████| 163/163 [04:29<00:00,  1.65s/it]

[[0.42775998 1.6701598 ]
 [0.7797613  2.5322058 ]
 [1.1943138  3.2235363 ]
 [1.6340176  3.8211486 ]
 [2.080834   4.4243484 ]
 [2.535665   5.026475  ]
 [2.9965143  5.677689  ]
 [3.452464   6.3041677 ]
 [3.9112203  6.9336143 ]
 [4.3745546  7.5578766 ]
 [4.840257   8.169381  ]
 [5.310988   8.750622  ]]


In [21]:
print(np.std(errors, axis=0))

[[0.59710056 3.186433  ]
 [1.0784663  4.0809026 ]
 [1.5833673  4.7050915 ]
 [2.0881422  5.2325544 ]
 [2.5937524  5.712854  ]
 [3.1028488  6.160838  ]
 [3.5992775  6.5368648 ]
 [4.095794   6.8820863 ]
 [4.5897317  7.2306066 ]
 [5.073547   7.5753455 ]
 [5.547339   7.9324613 ]
 [6.0154667  8.2943945 ]]
